## Testing dataset

In [1]:
import pandas as pd

df_ads = pd.read_csv('trial_200.csv')

# SELECT cc_text, ad FROM ads_nonads
df_ads = df_ads[["cc_text"]]

In [2]:
print(df_ads.head())
print(df_ads.shape)
print(df_ads.info())

                                             cc_text
0  Projects were rebuilding America and were incl...
1  Need to get your a1c down? You may pay as litt...
2  it's gonna help you and your relationship befo...
3  Of other countries. More analysis there, too, ...
4  Managing your diabetes just got easier. I'd sa...
(200, 1)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cc_text  200 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB
None


## Data Cleaning

In [3]:
# drop rows with any missing values
df_ads = df_ads.dropna()
print(df_ads.shape)
# drop duplicate rows
#df_ads = df_ads.drop_duplicates()
print(df_ads.shape)
# drop rows where 'ad' is not 0 or 1
#df_ads = df_ads[df_ads['labels'].isin([0, 1])]
print(df_ads.shape)

# Convert 'cc_text' column to string
df_ads['cc_text'] = df_ads['cc_text'].astype(str)
# Convert 'ad' column to integer
#df_ads['labels'] = df_ads['labels'].astype(int)


(200, 1)
(200, 1)
(200, 1)


In [12]:
# data check after cleaning
#print(df_ads["labels"].value_counts())

In [4]:
#new
import re

def clean_text(text):
    # Remove everything within HTML tags
    text = re.sub(r'<.*?>', '', text)
    # lower case 
    text = text.lower()
    # Remove special characters except for commas and periods
    text = re.sub(r'[^a-z\s,.]', '', text)
    # Remove special characters at the beginning of the sentence
    # text = re.sub(r'^[^A-Za-z0-9\s]+', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [5]:
import re

def clean_text(text):
    # Remove everything within HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters at the beginning of the sentence
    text = re.sub(r'^[^A-Za-z0-9\s]+', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


In [5]:
sample_1 = clean_text(df_ads["cc_text"][108])
print(sample_1)

and in the first hundred days in his office, he delivered something to you that hasnt been delivered to the cities of america since eisenhower was in office. thats a receipt that we never, ever going to forget. and as a consequence of all of this legislation, the president has created million jobs. we have the lowest unemployment rate in the history of the united states for the last years, the lowest unemployment rate for africanamericans, for hispanics. count it any way you want. this economy is blowing and going. but the president knows that this economy is not treating everybody equally. thats why hes going after big pharma. thats why hes going after the big oil and gas industries. thats why hes going after the big grocery store chains to not to quit profit taking and to let those costs go down, just like now. that costs because of supply chains have been lifted. and that is what president biden has spent his time doing. and hes going to continue to do that because it works. but, la

In [6]:
sample_2 = clean_text(df_ads["cc_text"][11])
print(sample_2)

teachers union. he is owned by the teachers union. governor newsom we will never cross dash. governor desantis lock, stop, and real. governor newsom lets talk about your record on covert on covert. he passed an emergency declaration. to close down your beaches, its bars, restaurants. it is a fact. you had quarantines. you had quarantines. you had checkpoints all over the state of florida. by the way, i did not say that. donald trump laid you out on this. you did that. he followed science. apollo tax. you follow science. governor desantis that is not true. governor newsom you follow tajik. you were promoting vaccines. sean we were open. you were close. governor newsom if it is okay with you overlapping speakers governor desantis wire you closed for so long sean im trying to let it breathe. governor newsom he did all of that until he decided to fall prey to the fringe of his party


In [7]:
# Apply clean_text function to cc_text column
df_ads["cc_text"] = df_ads["cc_text"].apply(clean_text)

In [8]:
df_ads.head()

,cc_text
0,projects were rebuilding america and were incl...
1,need to get your ac down you may pay as little...
2,its gonna help you and your relationship befor...
3,"of other countries. more analysis there, too, ..."
4,managing your diabetes just got easier. id say...


In [9]:
from transformers import pipeline
classifier = pipeline('text-classification', model="bert-base-uncased-adremoval_testingdata")

In [10]:
# Define a function to get predictions and scores
def get_predictions(text):
    result = classifier(text)
    return result[0]['label'], result[0]['score']

In [11]:
# Apply the function to each row in the DataFrame
df_ads['predicted_label'], df_ads['prediction_score'] = zip(*df_ads['cc_text'].apply(get_predictions))

In [12]:
df_ads.head()

,cc_text,predicted_label,prediction_score
0,projects were rebuilding america and were incl...,LABEL_0,0.691484
1,need to get your ac down you may pay as little...,LABEL_1,0.806898
2,its gonna help you and your relationship befor...,LABEL_0,0.632561
3,"of other countries. more analysis there, too, ...",LABEL_1,0.830386
4,managing your diabetes just got easier. id say...,LABEL_1,0.898113


In [14]:
df_ads.to_csv('trial.xlsx', index=False)

In [13]:
# Calculate accuracy metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Convert label strings to integers
df_ads['predicted_label'] = df_ads['predicted_label'].apply(lambda x: 1 if x == 'LABEL_1' else 0)

# Calculate accuracy
#accuracy = accuracy_score(df_ads['labels'], df_ads['predicted_label'])
#precision, recall, f1, _ = precision_recall_fscore_support(df_ads['labels'], df_ads['predicted_label'], average='binary')

#print(f"Accuracy: {accuracy}")
#print(f"Precision: {precision}")
#print(f"Recall: {recall}")
#print(f"F1 Score: {f1}")

In [15]:
# Identify mislabeled rows
mislabeled = df_ads[df_ads['labels'] != df_ads["predicted_label"]]

In [20]:
mislabeled.to_csv('mislabeled.csv', index=False)